In [1]:
from search import *
from game import *
from util import *
from pacman import *
import util, layout
import sys, types, time, random, os
import numpy as np
from random import random, randint, seed, choice


In [2]:

STATE_VARS = ['actionScore', 'eatFood', 'minGhostDist', 'minDistToFood', 'distToPill', 'minDistEdibleGhost']
" actionScore: scoreFuturo - scoreAtual "
" eatFood: numFoodAtual - numFoodFuturo"
" minGhostDist: |ghostX - pacmanX| + |ghostY - pacmanY| "
" minDistToFood: |foodX - pacmanX| + |foodY - pacmanY| "
" distToPill: |pillX - pacmanX| + |pillY - pacmanY| "


       

' distToPill: |pillX - pacmanX| + |pillY - pacmanY| '

In [3]:
" FITNESS "

# stateVars -> dictionary  ex stateVars['distX'] = distXGameStateVariable
def getFitness(individuo):
    return trainIndividual(individuo)

def getPopFitness(population):
    return [getFitness(population[i]) for i in range(0,len(population))]

    
    




In [10]:
n = Directions.NORTH
s = Directions.SOUTH
e = Directions.EAST
w = Directions.WEST

def manhattanDistance(posA, posB):
    return (abs(posB[0] - posA[0]) + abs(posB[1] - posA[1]))

def calcMinGhostDist(pacmanPosition, ghostsPosition):
    dist = 99999
    for ghost in ghostsPosition:
        if dist > manhattanDistance(pacmanPosition, ghost):
            dist = manhattanDistance(pacmanPosition, ghost)
            
    return dist

def manhattanFoodDistance(posA, posX, posY):
    return (abs(posX - posA[0]) + abs(posY - posA[1]))

def minDistBFS(agentPos, positions, wallGrid):
    # Toma paredes como celulas ja visitadas
    visited = wallGrid.copy()
    queue = []
    queue.append((agentPos,1))
    visited[agentPos[0]][agentPos[1]] = True
    if len(positions) > 0:
        while queue:
            el = queue.pop(0)
            h = el[1]
            s = el[0]

            north = (s[0],s[1]+1)
            south = (s[0],s[1]-1)
            west = (s[0]-1,s[1])
            east = (s[0]+1,s[1])

            for i in [north, south, west, east]:    
                if visited[i[0]][i[1]] == False:
                    queue.append((i,h+1))
                    visited[i[0]][i[1]] = True

                    if(i in positions):    
    #                     print (i, end = " ")
                        return h
                
    return 0

def findFoodMinDistBFS(agentPos, foodGrid, wallGrid):
    # Toma paredes como celulas ja visitadas
    visited = wallGrid.copy()
    queue = []
    queue.append((agentPos,1))
    visited[agentPos[0]][agentPos[1]] = True

    while queue:
        el = queue.pop(0)
        h = el[1]
        s = el[0]

        north = (s[0],s[1]+1)
        south = (s[0],s[1]-1)
        west = (s[0]-1,s[1])
        east = (s[0]+1,s[1])

        for i in [north, south, west, east]:    
            if visited[i[0]][i[1]] == False:
                queue.append((i,h+1))
                visited[i[0]][i[1]] = True

                if(foodGrid[i[0]][i[1]]):    
#                     print (i, end = " ")
                    return h
                
    return 0

def getMinDistEdibleGhost(agentPos,state):
    ghostPos = []
    ghostStates = state.getGhostStates()
    for gs in ghostStates:
        if gs.scaredTimer > 0:
            ghostPos.append(gs.getPosition())
            
    return minDistBFS(agentPos, ghostPos, state.getWalls())

# def calcFeatures(state, action):
#     futureState = state.generatePacmanSuccessor(action)
#     print("@@ ",state.getLegalActions(),"   -  ",futureState.getLegalActions())
            
#     futureScore = futureState.getScore() - state.getScore()
#     eatFood = state.getNumFood() - futureState.getNumFood()
#     ghostDist = calcMinGhostDist(futureState.getPacmanPosition(),futureState.getGhostPositions())
#     foodDist = findFoodMinDistBFS(futureState.getPacmanPosition(),futureState.getFood(),state.getWalls())
#     pillDist = calcMinPillDist(futureState.getPacmanPosition(),futureState.getCapsules())
# #             stateVars[0] = futureScore
# #             stateVars[1] = eatFood
# #             stateVars[2] = ghostDist
# #             stateVars[3] = foodDist
# #             stateVars[4] = pillDist
#     print("STATE VARS: FS: ", futureScore," EF: ", eatFood, " GD: ", ghostDist," FD: ", foodDist, " PD: ", pillDist)
#     return np.array([[futureScore/10,eatFood,ghostDist/10,foodDist/10,pillDist/10]])
directionsAux = {}
directionsAux['North'] = (0,+1)
directionsAux['South'] = (0,-1)
directionsAux['West'] = (-1,0)
directionsAux['East'] = (+1,0)
def calcFeatures(state, action):
    futureState = state.generatePacmanSuccessor(action)
#     print("@@ ",state.getLegalActions(),"   -  ")
    d = directionsAux[action]
    pos = state.getPacmanPosition()
    
    future_pos = (pos[0]+d[0],pos[1]+d[1])
    
    futureScore = futureState.getScore() - state.getScore()
    eatFood = 1.0 if state.getFood()[future_pos[0]][future_pos[1]] else 0.0
    ghostDist = minDistBFS(future_pos, state.getGhostPositions(), state.getWalls())
    foodDist = findFoodMinDistBFS(future_pos,state.getFood(),state.getWalls())
    pillDist = minDistBFS(future_pos, state.getCapsules(), state.getWalls())
    edibleGhost = getMinDistEdibleGhost(future_pos,state)

    print("STATE VARS:  FS: ", futureScore," EF: ", eatFood, " GD: ", ghostDist," FD: ", foodDist, " PD: ", pillDist," EG: ",edibleGhost)
    return np.array([[futureScore/20,eatFood,ghostDist/20,foodDist/20,pillDist/20,edibleGhost/20]])

class MyGoWestAgent(Agent):
    "An agent that goes West until it can't."
    def __init__(self, W = None, Q=0):
        self.W = W
        self.Q = Q
        
    def setW(self, W):
        self.W = W
        
    def setQ(self, Q):
        self.Q = Q

    def getAction(self, state):
        "The agent receives a GameState (defined in pacman.py)."
#         print(state.getLegalPacmanActions())
#         print(state.getPacmanState())

        print("PACMANPOS0 :", state.getPacmanPosition())
#         print(state.getNumFood())
#         print(state.getScore())
        count = 0
        legalActions = state.getLegalPacmanActions()
        legalActions.remove(Directions.STOP) #remove a opcao Stop
        actionScore = []
        
        for action in legalActions:    
            stateVars = calcFeatures(state, action)
            actionValue = np.dot(self.W,stateVars.T)
            actionScore.append(actionValue)
            
        print("Action Score: ",actionScore)
        print(state.getLegalPacmanActions())
        if actionScore.count(max(actionScore)) > 1:
            maxScore = max(actionScore)
            indexes = [i for i, j in enumerate(actionScore) if j == maxScore]
            maxIndex = choice(indexes)
        else:
            maxIndex = actionScore.index(max(actionScore))
            
        action = legalActions[maxIndex]
        
        gama = 0.9
        r = 0
        alfa = 0.1
        
        difference = (r+gama*max(actionScore)) - self.Q
        self.setQ(max(actionScore))
#         print(state.getLegalPacmanActions()," ACTION: ",action)
        self.setW(self.W + alfa*difference*calcFeatures(state, action))
        
        print("W: ", self.W)
#         print("Action: ",action)
#         futureState = state.generatePacmanSuccessor(action)
#         futureScore = futureState.getScore() - state.getScore()
#         eatFood = state.getNumFood() - futureState.getNumFood()
#         ghostDist = calcMinGhostDist(futureState.getPacmanPosition(),futureState.getGhostPositions())
#         foodDist = calcMinFoodDist(futureState.getPacmanPosition(),futureState.getFood())
#         print("STATE VARS: FS: ", futureScore," EF: ", eatFood, " GD: ", ghostDist," FD: ", foodDist, " PD: ", pillDist)
        
        if action in state.getLegalPacmanActions():
            return action
        else:
            return Directions.STOP

In [12]:
layoutType='smallClassic'
numGames=10
gameLayout = layout.getLayout( layoutType )
print("------------------------------------------ train --------------------")

W = np.zeros((1,6))
W[:] = 0.7
pacman = MyGoWestAgent(W)

ghostType = loadAgent('RandomGhost', True)
ghosts = [ghostType( i+1 ) for i in range( 4 )]

# import textDisplay
# textDisplay.SLEEP_TIME = 0.1
# gameDisplay = textDisplay.PacmanGraphics()
import graphicsDisplay
gameDisplay = graphicsDisplay.PacmanGraphics(1.0, frameTime = 0.1)

games = runGames( gameLayout, pacman, ghosts, gameDisplay, numGames, False)
scores = [game.state.getScore() for game in games]
wins = [game.state.isWin() for game in games]
winRate = wins.count(True)/ float(len(wins))
avgScore = sum(scores) / float(len(scores))

print(pacman.W)
print(avgScore)

layout:  smallClassic
------------------------------------------ train --------------------
PACMANPOS0 : (9, 1)
STATE VARS:  FS:  9.0  EF:  1.0  GD:  11  FD:  1  PD:  8  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  10  FD:  1  PD:  7  EG:  0
Action Score:  [array([[1.715]]), array([[1.645]])]
['East', 'West', 'Stop']
STATE VARS:  FS:  9.0  EF:  1.0  GD:  11  FD:  1  PD:  8  EG:  0
W:  [[0.7694575 0.85435   0.7848925 0.7077175 0.76174   0.7      ]]
PACMANPOS0 : (10, 1)
STATE VARS:  FS:  9.0  EF:  1.0  GD:  9  FD:  1  PD:  7  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  10  FD:  1  PD:  8  EG:  0
Action Score:  [array([[1.85580237]]), array([[0.69405525]])]
['East', 'West', 'Stop']
STATE VARS:  FS:  9.0  EF:  1.0  GD:  9  FD:  1  PD:  7  EG:  0
W:  [[0.7674425  0.84987221 0.7828775  0.70749361 0.76017277 0.7       ]]
PACMANPOS0 : (11, 1)
STATE VARS:  FS:  9.0  EF:  1.0  GD:  8  FD:  1  PD:  6  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  10  FD:  2  PD:  8  EG:  0
Action Score:  [array

PACMANPOS0 : (11, 1)
STATE VARS:  FS:  9.0  EF:  1.0  GD:  7  FD:  1  PD:  6  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  9  FD:  2  PD:  8  EG:  0
Action Score:  [array([[1.21876475]]), array([[0.5863308]])]
['East', 'West', 'Stop']
STATE VARS:  FS:  9.0  EF:  1.0  GD:  7  FD:  1  PD:  6  EG:  0
W:  [[0.59564532 0.46810072 0.60979701 0.68840504 0.66614159 0.7       ]]
PACMANPOS0 : (12, 1)
STATE VARS:  FS:  9.0  EF:  1.0  GD:  5  FD:  1  PD:  5  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  7  FD:  2  PD:  7  EG:  0
Action Score:  [array([[1.08954601]]), array([[0.48563675]])]
['East', 'West', 'Stop']
STATE VARS:  FS:  9.0  EF:  1.0  GD:  5  FD:  1  PD:  5  EG:  0
W:  [[0.58492752 0.44428338 0.60384267 0.68721417 0.66018726 0.7       ]]
PACMANPOS0 : (13, 1)
STATE VARS:  FS:  9.0  EF:  1.0  GD:  3  FD:  1  PD:  6  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  5  FD:  1  PD:  4  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  5  FD:  2  PD:  6  EG:  0
Action Score:  [array([[1.03049405]])

PACMANPOS0 : (11, 3)
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  4  FD:  2  PD:  6  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  2  FD:  1  PD:  8  EG:  0
Action Score:  [array([[0.33491707]]), array([[0.77619248]])]
['East', 'West', 'Stop']
STATE VARS:  FS:  9.0  EF:  1.0  GD:  2  FD:  1  PD:  8  EG:  0
W:  [[0.48354987 0.21899971 0.55775778 0.67594999 0.59642589 0.7       ]]
PACMANPOS0 : (10, 3)
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  1  FD:  2  PD:  9  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  3  FD:  2  PD:  7  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  3  FD:  1  PD:  8  EG:  0
Action Score:  [array([[0.33969705]]), array([[0.33583023]]), array([[0.79262867]])]
['North', 'East', 'West', 'Stop']
STATE VARS:  FS:  9.0  EF:  1.0  GD:  3  FD:  1  PD:  8  EG:  0
W:  [[0.48072267 0.21271704 0.55681538 0.67563585 0.59391283 0.7       ]]
PACMANPOS0 : (9, 3)
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  2  FD:  2  PD:  9  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  2  FD:  2  PD:  8  EG:  0
STA

PACMANPOS0 : (7, 1)
STATE VARS:  FS:  -501.0  EF:  0.0  GD:  7  FD:  3  PD:  7  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  2  FD:  1  PD:  5  EG:  0
Action Score:  [array([[-11.20270339]]), array([[0.19432093]])]
['East', 'West', 'Stop']
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  2  FD:  1  PD:  5  EG:  0
W:  [[0.46617108 0.16632335 0.51432576 0.65054718 0.53220872 0.7       ]]
PACMANPOS0 : (6, 1)
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  2  FD:  2  PD:  6  EG:  0
STATE VARS:  FS:  -501.0  EF:  0.0  GD:  7  FD:  2  PD:  6  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  2  FD:  1  PD:  4  EG:  0
Action Score:  [array([[0.25284135]]), array([[-11.27285427]]), array([[0.56650202]])]
['North', 'East', 'West', 'Stop']
STATE VARS:  FS:  9.0  EF:  1.0  GD:  2  FD:  1  PD:  4  EG:  0
W:  [[0.48036997 0.19787644 0.51748107 0.65212484 0.53851933 0.7       ]]
PACMANPOS0 : (5, 1)
STATE VARS:  FS:  -501.0  EF:  0.0  GD:  7  FD:  2  PD:  5  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  2  FD:  1  PD:  3  E

PACMANPOS0 : (4, 4)
STATE VARS:  FS:  9.0  EF:  1.0  GD:  0  FD:  1  PD:  16  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  0  FD:  2  PD:  16  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  0  FD:  1  PD:  14  EG:  0
Action Score:  [array([[0.89991729]]), array([[0.46237062]]), array([[0.84708861]])]
['North', 'South', 'East', 'Stop']
STATE VARS:  FS:  9.0  EF:  1.0  GD:  0  FD:  1  PD:  16  EG:  0
W:  [[0.48581103 0.20064252 0.51326488 0.64406838 0.51114023 0.7101686 ]]
PACMANPOS0 : (4, 5)
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  4  FD:  1  PD:  15  EG:  4
STATE VARS:  FS:  9.0  EF:  1.0  GD:  4  FD:  1  PD:  15  EG:  4
STATE VARS:  FS:  9.0  EF:  1.0  GD:  3  FD:  1  PD:  17  EG:  3
Action Score:  [array([[0.63595474]]), array([[1.07950278]]), array([[1.06944513]])]
['South', 'East', 'West', 'Stop']
STATE VARS:  FS:  9.0  EF:  1.0  GD:  4  FD:  1  PD:  15  EG:  4
W:  [[0.48903461 0.20780605 0.51469759 0.64442656 0.51651288 0.7116013 ]]
PACMANPOS0 : (5, 5)
STATE VARS:  FS:  9.0  EF:  1.0

...

PACMANPOS0 : (6, 4)
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  5  FD:  3  PD:  5  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  3  FD:  1  PD:  5  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  5  FD:  3  PD:  3  EG:  0
Action Score:  [array([[-2.62555698]]), array([[-2.34710861]]), array([[-2.39938604]])]
['North', 'South', 'West', 'Stop']
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  3  FD:  1  PD:  5  EG:  0
W:  [[28.61891318 -5.4578779  -2.11043161 -0.66870051 -2.25501999  0.71257382]]
PACMANPOS0 : (6, 3)
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  3  FD:  2  PD:  4  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  1  FD:  2  PD:  6  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  3  FD:  1  PD:  6  EG:  0
Action Score:  [array([[-2.26538445]]), array([[-2.27984329]]), array([[6.39412727]])]
['North', 'South', 'East', 'Stop']
STATE VARS:  FS:  9.0  EF:  1.0  GD:  3  FD:  1  PD:  6  EG:  0
W:  [[28.98349522 -4.64769558 -1.98890426 -0.62819139 -2.01196529  0.71257382]]
PACMANPOS0 : (7, 3)
STATE VARS:  FS: 

PACMANPOS0 : (4, 4)
STATE VARS:  FS:  9.0  EF:  1.0  GD:  2  FD:  1  PD:  3  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  2  FD:  1  PD:  1  EG:  0
STATE VARS:  FS:  -501.0  EF:  0.0  GD:  1  FD:  1  PD:  3  EG:  0
Action Score:  [array([[6.35365876]]), array([[6.58613879]]), array([[-713.16713059]])]
['North', 'South', 'East', 'Stop']
STATE VARS:  FS:  9.0  EF:  1.0  GD:  2  FD:  1  PD:  1  EG:  0
W:  [[28.42203108 -5.89539368 -2.32646106 -0.6905763  -2.32788805  0.71257382]]
PACMANPOS0 : (4, 3)
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  1  FD:  1  PD:  2  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  3  FD:  1  PD:  2  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  3  FD:  2  PD:  17  EG:  0
Action Score:  [array([[-1.80474223]]), array([[6.27823352]]), array([[-3.81783318]])]
['North', 'South', 'West', 'Stop']
STATE VARS:  FS:  9.0  EF:  1.0  GD:  3  FD:  1  PD:  2  EG:  0
W:  [[28.37992329 -5.98896655 -2.34049699 -0.69525494 -2.33724533  0.71257382]]
PACMANPOS0 : (4, 2)
STATE VARS:  FS: 

PACMANPOS0 : (4, 2)
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  3  FD:  2  PD:  1  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  1  FD:  4  PD:  3  EG:  0
Action Score:  [array([[-2.0065704]]), array([[-2.10345893]])]
['North', 'South', 'Stop']
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  3  FD:  2  PD:  1  EG:  0
W:  [[28.60188391 -5.64262689 -2.44809133 -0.75870295 -2.65827371  0.71257382]]
PACMANPOS0 : (4, 3)
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  5  FD:  1  PD:  2  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  3  FD:  3  PD:  2  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  5  FD:  3  PD:  17  EG:  0
Action Score:  [array([[-2.34587955]]), array([[-2.17694071]]), array([[-4.41545512]])]
['North', 'South', 'West', 'Stop']
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  3  FD:  3  PD:  2  EG:  0
W:  [[28.60164729 -5.64262689 -2.44738147 -0.75799309 -2.65780047  0.71257382]]
PACMANPOS0 : (4, 2)
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  5  FD:  2  PD:  1  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  3  FD: 

PACMANPOS0 : (7, 3)
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  2  FD:  2  PD:  7  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  4  FD:  4  PD:  5  EG:  0
Action Score:  [array([[-2.76713987]]), array([[-2.79692973]])]
['East', 'West', 'Stop']
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  2  FD:  2  PD:  7  EG:  0
W:  [[29.43869146 -3.97984845 -2.2465561  -0.85963861 -2.80556832  0.71257382]]
PACMANPOS0 : (8, 3)
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  1  FD:  1  PD:  8  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  3  FD:  3  PD:  6  EG:  0
Action Score:  [array([[-2.74947163]]), array([[-2.77953427]])]
['East', 'West', 'Stop']
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  1  FD:  1  PD:  8  EG:  0
W:  [[29.43722838 -3.97984845 -2.24509302 -0.85817553 -2.7938637   0.71257382]]
PACMANPOS0 : (9, 3)
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  1  FD:  2  PD:  9  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  1  FD:  1  PD:  8  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  3  FD:  2  PD:  7  EG:  0
Action Score:  [ar

PACMANPOS0 : (5, 1)
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  7  FD:  1  PD:  5  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  9  FD:  1  PD:  3  EG:  0
Action Score:  [array([[-3.12719812]]), array([[7.08085855]])]
['East', 'West', 'Stop']
STATE VARS:  FS:  9.0  EF:  1.0  GD:  9  FD:  1  PD:  3  EG:  0
W:  [[29.21075842 -4.48311504 -2.73647728 -0.88333886 -2.70832736  0.71257382]]
PACMANPOS0 : (4, 1)
STATE VARS:  FS:  9.0  EF:  1.0  GD:  10  FD:  1  PD:  2  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  8  FD:  2  PD:  4  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  10  FD:  1  PD:  4  EG:  0
Action Score:  [array([[6.97848793]]), array([[-3.18512819]]), array([[6.70765519]])]
['North', 'East', 'West', 'Stop']
STATE VARS:  FS:  9.0  EF:  1.0  GD:  10  FD:  1  PD:  2  EG:  0
W:  [[29.17474854 -4.56313698 -2.77648825 -0.88733996 -2.71632955  0.71257382]]
PACMANPOS0 : (4, 2)
STATE VARS:  FS:  9.0  EF:  1.0  GD:  8  FD:  1  PD:  1  EG:  0
STATE VARS:  FS:  -1.0  EF:  0.0  GD:  8  FD:  1  PD:  

In [6]:
# GAME LAYOUTS: smallClassic, mediumClassic, originalClassic
gameLayout = layout.getLayout( 'mediumClassic' )

ind = GPTree()
ind.random_tree(True, 2)
ind.print_tree()

pacman = MyGoWestAgent(ind)
# pacman.setTree(ind)

ghostType = loadAgent('RandomGhost', True)
ghosts = [ghostType( i+1 ) for i in range( 4 )]

import textDisplay
textDisplay.SLEEP_TIME = 0.1
gameDisplay = textDisplay.PacmanGraphics()

runGames( gameLayout, pacman, ghosts, gameDisplay, 4, False)

layout:  mediumClassic


NameError: name 'GPTree' is not defined